In [ ]:
!pip install plotly
!pip install dash
!pip install seaborn matplotlib
!pip install scipy
!pip install plotly dash seaborn matplotlib scipy gdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 17.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import gdown as gd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import dash
from dash import dcc, html
import plotly.express as px



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

file_path = '/content/drive/My Drive/Phân tích DL/Dữ liệu/dữ liệu qua xử lý.csv' # Thay đổi đường dẫn đến tệp CSV của bạn
df = pd.read_csv(file_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.tail()

,pc_brand,pc_model,elt_condition,laptop_screen_size,elt_warranty,pc_cpu,pc_ram,pc_drive_capacity,elt_origin,pc_vga,price
934,Hãng khác,Dòng Khác,Đã sử dụng (chưa sửa chữa),15 - 16.9 inch,1 tháng,Intel Core i5,4,500,Việt Nam,Onboard,2490000
935,Lenovo,ThinkPad,Đã sử dụng (chưa sửa chữa),13 - 14.9 inch,4-6 tháng,Intel Core i7,16,256,Mỹ,NVIDIA,8600000
936,Apple,MacBook Pro,Đã sử dụng (chưa sửa chữa),15 - 16.9 inch,3 tháng,Intel Core i7,16,256,Mỹ,AMD,14500000
937,Dell,Precision,Mới,15 - 16.9 inch,3 tháng,Intel Core i7,16,512,Mỹ,NVIDIA,14900000
938,Apple,MacBook Pro,Đã sử dụng (chưa sửa chữa),13 - 14.9 inch,1 tháng,Intel Core i5,8,128,Mỹ,Khác,8390000


In [ ]:
df.isnull().sum()

pc_brand              0
pc_model              0
elt_condition         0
laptop_screen_size    0
elt_warranty          0
pc_cpu                0
pc_ram                0
pc_drive_capacity     0
elt_origin            0
pc_vga                0
price                 0
dtype: int64

In [ ]:
num_rows = df.shape[0]
print("Số lượng dòng trong dữ liệu là:", num_rows)

Số lượng dòng trong dữ liệu là: 939


In [ ]:
df.dtypes

pc_brand              object
pc_model              object
elt_condition         object
laptop_screen_size    object
elt_warranty          object
pc_cpu                object
pc_ram                 int64
pc_drive_capacity      int64
elt_origin            object
pc_vga                object
price                  int64
dtype: object

1. PHÂN TÍCH DỮ LIỆU



 Phân loại các biến số trong bộ dữ liệu biến nào tương đối gần mức đối xứng, các biến nào đang mất cân đố

In [ ]:
df2 = df.copy()
def KiemTraDoiXung(data: pd.Series, percent: float):
    median = data.median()
    mean = data.mean()
    mode = data.mode()


    if len(mode) == 1:
        if abs(median - mean) / median < percent / 100:
            return True
    return False
def KiemTraCacBien(ds: pd.DataFrame):
    d = {}
    numeric_columns = ds.select_dtypes(include=['int64', 'float64',]).columns
    for col in numeric_columns:
        d[col] = KiemTraDoiXung(ds[col], 10)
    return d
KiemTraCacBien(df2)


{'pc_ram': False, 'pc_drive_capacity': False, 'price': False}

Máy tính dạng nào (hãng nào, cpu, ram) sẽ có giá tốt nhất

In [ ]:
# Lọc các cột quan trọng
selected_columns = ['pc_brand', 'pc_model', 'elt_condition', 'laptop_screen_size', 'elt_warranty',
                    'pc_cpu', 'pc_ram', 'pc_drive_capacity', 'elt_origin', 'pc_vga', 'price']
df_selected = df[selected_columns]

# Sắp xếp theo giá từ cao đến thấp
df_sorted = df_selected.sort_values(by='price', ascending=False)

# Lấy ra máy tính có giá cao nhất
most_expensive_deal = df_sorted.iloc[0]

# Hiển thị thông tin về máy tính có giá cao nhất
print("Thông tin về máy tính có giá cao nhất:")
print(most_expensive_deal[['pc_brand', 'pc_cpu', 'pc_ram']])


Thông tin về máy tính có giá cao nhất:
pc_brand             Dell
pc_cpu      Intel Core i9
pc_ram                 32
Name: 428, dtype: object


Biến số nào ảnh hưởng đến giá

In [ ]:
# Phân tích tương quan giữa pc_ram và giá
correlation_ram_price, p_value_ram_price = stats.pearsonr(df['pc_ram'], df['price'])
print(f"Correlation between pc_ram and price: {correlation_ram_price}")
print(f"P-value: {p_value_ram_price}")

# Phân tích tương quan giữa pc_drive_capacity và giá
correlation_capacity_price, p_value_capacity_price = stats.pearsonr(df['pc_drive_capacity'], df['price'])
print(f"Correlation between pc_drive_capacity and price: {correlation_capacity_price}")
print(f"P-value: {p_value_capacity_price}")

Correlation between pc_ram and price: 0.5977359222603316
P-value: 4.945733968804066e-92
Correlation between pc_drive_capacity and price: 0.5905929968466154
P-value: 2.3386863216227973e-89


Trong các biến phân loại trên thì biến nào ảnh hưởng đến giá cả

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Tạo mô hình ANOVA
model = ols('price ~ pc_brand + pc_model + elt_condition + laptop_screen_size + pc_cpu + elt_warranty + elt_origin + pc_vga', data=df).fit()

# Phân tích ANOVA
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)

                          sum_sq     df          F        PR(>F)
pc_brand            6.483358e+14   13.0   3.020329  2.307181e-04
pc_model            1.019904e+16   73.0   8.461238  4.567026e-61
elt_condition       4.021004e+14    2.0  12.175929  6.151311e-06
laptop_screen_size  3.987727e+14    4.0   6.037582  8.635874e-05
pc_cpu              5.393866e+15   13.0  25.127794  1.700012e-51
elt_warranty        1.850871e+15    8.0  14.011470  3.268678e-19
elt_origin          1.127839e+14    6.0   1.138396  3.379428e-01
pc_vga              3.046923e+14    3.0   6.150886  3.888482e-04
Residual            1.355644e+16  821.0        NaN           NaN


In [ ]:
# Kiểm tra giá trị p-value của từng biến phân loại
p_values = anova_table['PR(>F)']
significant_variables = [variable for variable, p_value in zip(anova_table.index, p_values) if p_value < 0.05]

# In ra biến phân loại ảnh hưởng đáng kể đến giá cả
print("Các biến phân loại ảnh hưởng đáng kể đến giá cả:", significant_variables)

Các biến phân loại ảnh hưởng đáng kể đến giá cả: ['pc_brand', 'pc_model', 'elt_condition', 'laptop_screen_size', 'pc_cpu', 'elt_warranty', 'pc_vga']


Tập hợp gồm 3 giá trị biến phân loại có khả năng ảnh hưởng đến giá nhiều nhất:

In [ ]:
categorical_columns = df.select_dtypes(include=['object'])
top_3_categorical = {}

# Duyệt qua từng cột phân loại và tính F-statistic và p-value
for column in categorical_columns:
    groups = df.groupby(column)['price'].apply(list)
    f_statistic, p_value = stats.f_oneway(*groups)
    top_3_categorical[column] = {'F-statistic': f_statistic, 'p-value': p_value}

# Sắp xếp tập hợp theo giá trị F-statistic và lấy 3 giá trị cao nhất
sorted_top_3_categorical = sorted(top_3_categorical.items(), key=lambda item: item[1]['F-statistic'], reverse=True)[:3]

# Xuất ra kết quả
print("Tìm tập hợp gồm 3 giá trị biến phân loại có khả năng ảnh hưởng đến giá nhiều nhất:")
for column, values in sorted_top_3_categorical:
    f_statistic = values['F-statistic']
    p_value = values['p-value']
    print(f"{column} Với F-statistic = {f_statistic}, p-value = {p_value}")

Tìm tập hợp gồm 3 giá trị biến phân loại có khả năng ảnh hưởng đến giá nhiều nhất:
pc_cpu Với F-statistic = 50.51420968547747, p-value = 2.3485345047051534e-98
elt_condition Với F-statistic = 48.63758778691589, p-value = 8.016988701364902e-21
pc_vga Với F-statistic = 30.7681370933067, p-value = 5.677418484231058e-19


Sự biến thiên về giá do ảnh hưởng của các biến

In [ ]:
import statsmodels.api as sm

# Chọn 11 biến cần tính ảnh hưởng
selected_variables = [
    "pc_brand", "pc_model", "elt_condition", "laptop_screen_size",
    "elt_warranty", "pc_cpu", "pc_ram", "pc_drive_capacity",
    "elt_origin", "pc_vga"
]

# Tạo DataFrame mới chỉ chứa các biến được chọn và giá
df_selected = df[selected_variables + ['price']]

# Xử lý dữ liệu categorical thành dummies
df_selected = pd.get_dummies(df_selected, drop_first=True)

# Thêm hằng số cho mô hình Linear Regression
X = sm.add_constant(df_selected.drop('price', axis=1))

# Mô hình Linear Regression
model = sm.OLS(df_selected['price'], X).fit()

# In summary của mô hình
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.834
Model:                            OLS   Adj. R-squared:                  0.810
Method:                 Least Squares   F-statistic:                     34.61
Date:                Wed, 20 Dec 2023   Prob (F-statistic):          1.70e-249
Time:                        02:15:51   Log-Likelihood:                -15430.
No. Observations:                 939   AIC:                         3.110e+04
Df Residuals:                     819   BIC:                         3.168e+04
Df Model:                         119                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

2. TRỰC QUAN HÓA DỮ LIỆU

Nội dung
1.   Đồ thị thể hiện sự phân phối giá bán máy tính
2.   Ma trận tương quan giữa các biến trong dữ liệu
3.   Số lượng máy tính của hãng được bán nhiều nhất
4.   Số lượng từng dòng máy của hãng được bán nhiều nhất
5.   Boxplot ảnh hưởng của pc-drive-capacity đối với giá
6.   Boxplot ảnh hưởng của pc-ram đối với giá


In [ ]:
# Tính toán ma trận tương quan
correlation_matrix = df.corr()

# In ma trận tương quan để kiểm tra
print(correlation_matrix)

                     pc_ram  pc_drive_capacity     price
pc_ram             1.000000           0.541672  0.597736
pc_drive_capacity  0.541672           1.000000  0.590593
price              0.597736           0.590593  1.000000


<ipython-input-17-5582f3c5b19d>:2: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



In [ ]:
import numpy as np
# Đếm số lượng sản phẩm bán theo từng hãng máy tính
brand_counts = df['pc_brand'].value_counts()

# Đếm số lượng từng dòng máy thuộc hãng Dell
dell_counts = df[df['pc_brand'] == 'Dell']['pc_model'].value_counts()

app = dash.Dash(__name__)

app.layout = html.Div(children=[
    html.H1(children='Dashboard Phân tích dữ liệu'),
    html.Div([
        dcc.Graph(
            id='histogram-price',
            figure={
                'data': [
                    {'x': df['price'], 'type': 'histogram', 'name': 'Phân phối giá'}
                ],
                'layout': {
                    'title': 'Phân phối giá'
                }
            }
        ),
        dcc.Graph(
            id='heatmap-correlation',
            figure={
                'data': [
                    {'z': correlation_matrix.values,
                     'x': correlation_matrix.columns,
                     'y': correlation_matrix.index,
                     'type': 'heatmap',
                     'colorscale': 'coolwarm',
                     'colorbar': {'title': 'Tương quan'}
                     }
                ],
                'layout': {
                    'title': 'Ma trận Tương quan'
                }
            }
        ),
    ], style={'display': 'flex'}),

    html.Div([
        dcc.Graph(
            id='bar-chart',
            figure=px.bar(x=brand_counts.index, y=brand_counts.values, labels={'x': 'Hãng máy tính', 'y': 'Số lượng'}),
        ),
        dcc.Graph(
            id='pie-chart',
            figure=px.pie(labels=dell_counts.index, values=dell_counts.values,
                          title='Phân phối số lượng máy',
                          hover_name=dell_counts.index),
        ),
    ], style={'display': 'flex'}),

    html.Div([
        dcc.Graph(
            id='boxplot-drive-capacity-price',
            figure=px.box(df, x='pc_drive_capacity', y='price', title='Ảnh hưởng của pc_drive_capacity đối với giá'),
        ),
        dcc.Graph(
            id='boxplot-ram-price',
            figure=px.box(df, x='pc_ram', y='price', title='Ảnh hưởng của pc_ram đối với giá'),
        ),
    ], style={'display': 'flex'}),

])

if __name__ == '__main__':
    app.run_server(debug=True)


/usr/local/lib/python3.10/dist-packages/plotly/express/_core.py:137: FutureWarning:

Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.



<IPython.core.display.Javascript object>

3. MÔ HÌNH DỰ ĐOÁN KẾT QUẢ

XGboost

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import xgboost as xgb

# Chuẩn bị dữ liệu văn bản
X = df[['pc_brand', 'pc_model', 'elt_condition', 'laptop_screen_size', 'elt_warranty', 'pc_cpu', 'pc_vga', 'pc_ram', 'pc_drive_capacity']]
y = df['price']

# Chuyển đổi cột thành chuỗi duy nhất
X_text = X.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

# Chuyển đổi văn bản thành vectơ TF-IDF
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(X_text)

# Chia tập train và tập test
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.3, random_state=42)

# Xây dựng mô hình RandomForestRegressor
xgb_model = xgb.XGBRegressor()
xgb_model.fit(X_train, y_train)

# Huấn luyện mô hình
# rf_model.fit(X_train, y_train)

# Đánh giá mô hình
accuracy = xgb_model.score(X_test, y_test)
print("Độ chính xác của mô hình là:", accuracy)

Độ chính xác của mô hình là: 0.5630450689881404


K-Neighbors Regressor(KNN)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

# Chuẩn bị dữ liệu văn bản
X = df[['pc_brand', 'pc_model', 'elt_condition', 'laptop_screen_size', 'elt_warranty', 'pc_cpu', 'pc_vga', 'pc_ram', 'pc_drive_capacity']]
y = df['price']

# Chuyển đổi cột thành chuỗi duy nhất
X_text = X.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

# Chuyển đổi văn bản thành vectơ TF-IDF
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(X_text)

# Chia tập train và tập test
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.3, random_state=42)

# Xây dựng mô hình KNN
knn_model = KNeighborsRegressor()

# Huấn luyện mô hình
knn_model.fit(X_train, y_train)

# Đánh giá mô hình
accuracy = knn_model.score(X_test, y_test)
print("Độ chính xác của mô hình là:", accuracy)

Độ chính xác của mô hình là: 0.5737086094651964


Decision Tree Regressor

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

# Chuẩn bị dữ liệu văn bản
X = df[['pc_brand', 'pc_model', 'elt_condition', 'laptop_screen_size', 'elt_warranty', 'pc_cpu', 'pc_vga', 'pc_ram', 'pc_drive_capacity']]
y = df['price']

# Chuyển đổi cột thành chuỗi duy nhất
X_text = X.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

# Chuyển đổi văn bản thành vectơ TF-IDF
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(X_text)

# Chia tập train và tập test
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.3, random_state=42)

# Xây dựng mô hình Decision Tree
dt_model = DecisionTreeRegressor()

# Huấn luyện mô hình
dt_model.fit(X_train, y_train)

# Đánh giá mô hình
accuracy = dt_model.score(X_test, y_test)
print("Độ chính xác của mô hình là:", accuracy)

Độ chính xác của mô hình là: 0.5023753338496895


Random Forest Regressor

In [ ]:
# RandomForestRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Chuẩn bị dữ liệu văn bản
X = df[['pc_brand', 'pc_model', 'elt_condition', 'laptop_screen_size', 'elt_warranty', 'pc_cpu', 'pc_vga', 'pc_ram', 'pc_drive_capacity']]
y = df['price']

# Chuyển đổi cột thành chuỗi duy nhất
X_text = X.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

# Chuyển đổi văn bản thành vectơ TF-IDF
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(X_text)

# Chia tập train và tập test
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.3, random_state=42)

# Xây dựng mô hình RandomForestRegressor
rf_model = RandomForestRegressor()

# Huấn luyện mô hình
rf_model.fit(X_train, y_train)

# Đánh giá mô hình
accuracy = rf_model.score(X_test, y_test)
print("Độ chính xác của mô hình là:", accuracy)

Độ chính xác của mô hình là: 0.6690198231326057


In [ ]:
# Dự đoán thử
new_data = [['Dell', 'Latitude', 'Đã sử dụng (chưa sửa chữa)', '13 - 14.9 inch', '1 tháng', 'Intel Core i5', 'Onboard']]
new_text = pd.DataFrame(new_data).apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
new_tfidf = vectorizer.transform(new_text)
prediction = rf_model.predict(new_tfidf)

print("Giá dự đoán cho các giá trị đầu vào là:", prediction)

Giá dự đoán cho các giá trị đầu vào là: [6014331.83143114]


Không dùg Tfidf

In [ ]:
# RandomForest
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder

data=df

# Chuyển đổi thuộc tính văn bản thành dạng số bằng LabelEncoder
label_encoder = LabelEncoder()
data['pc_brand'] = label_encoder.fit_transform(data['pc_brand'])
data['pc_model'] = label_encoder.fit_transform(data['pc_model'])
data['elt_condition'] = label_encoder.fit_transform(data['elt_condition'])
data['laptop_screen_size'] = label_encoder.fit_transform(data['laptop_screen_size'])
data['elt_warranty'] = label_encoder.fit_transform(data['elt_warranty'])
data['pc_cpu'] = label_encoder.fit_transform(data['pc_cpu'])
data['pc_vga'] = label_encoder.fit_transform(data['pc_vga'])

# Tách thành features (đặc trưng) và target (giá)
data = data.drop(['elt_origin'], axis=1)
X = data.drop('price', axis=1)
y = data['price']

# Tách dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


# Huấn luyện mô hình Random Forest
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Dự đoán giá trên tập kiểm tra
y_pred = model.predict(X_test)

# Đánh giá hiệu suất mô hình
r2 = r2_score(y_test, y_pred)

print(f"Coefficient of Determination (R^2): {r2}")

Coefficient of Determination (R^2): 0.6119957565849351


In [ ]:
# xgboost
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder

data = df

# Chuyển đổi thuộc tính văn bản thành dạng số bằng LabelEncoder
label_encoder = LabelEncoder()
data['pc_brand'] = label_encoder.fit_transform(data['pc_brand'])
data['pc_model'] = label_encoder.fit_transform(data['pc_model'])
data['elt_condition'] = label_encoder.fit_transform(data['elt_condition'])
data['laptop_screen_size'] = label_encoder.fit_transform(data['laptop_screen_size'])
data['elt_warranty'] = label_encoder.fit_transform(data['elt_warranty'])
data['pc_cpu'] = label_encoder.fit_transform(data['pc_cpu'])
data['pc_vga'] = label_encoder.fit_transform(data['pc_vga'])

# Tách thành features (đặc trưng) và target (giá)
data = data.drop(['elt_origin'], axis=1)
X = data.drop('price', axis=1)
y = data['price']

# Tách dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Huấn luyện mô hình XGBoost
model = xgb.XGBRegressor()
model.fit(X_train, y_train)

# Dự đoán giá trên tập kiểm tra
y_pred = model.predict(X_test)

# Đánh giá hiệu suất mô hình
r2 = r2_score(y_test, y_pred)

print(f"Coefficient of Determination (R^2): {r2}")

Coefficient of Determination (R^2): 0.6929480426478298


In [ ]:
# DecisionTree
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder

data = df

# Tiền xử lý dữ liệu
# Chuyển đổi thuộc tính văn bản thành dạng số bằng LabelEncoder
label_encoder = LabelEncoder()
data['pc_brand'] = label_encoder.fit_transform(data['pc_brand'])
data['pc_model'] = label_encoder.fit_transform(data['pc_model'])
data['elt_condition'] = label_encoder.fit_transform(data['elt_condition'])
data['laptop_screen_size'] = label_encoder.fit_transform(data['laptop_screen_size'])
data['elt_warranty'] = label_encoder.fit_transform(data['elt_warranty'])
data['pc_cpu'] = label_encoder.fit_transform(data['pc_cpu'])
data['pc_vga'] = label_encoder.fit_transform(data['pc_vga'])

# Tách thành features (đặc trưng) và target (giá)
data = data.drop(['elt_origin'], axis=1)
X = data.drop('price', axis=1)
y = data['price']

# Tách dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Huấn luyện mô hình Decision Tree
model = DecisionTreeRegressor()
model.fit(X_train, y_train)

# Dự đoán giá trên tập kiểm tra
y_pred = model.predict(X_test)

# Đánh giá hiệu suất mô hình
r2 = r2_score(y_test, y_pred)

print(f"Coefficient of Determination (R^2): {r2}")

Coefficient of Determination (R^2): 0.22565697580777289


In [ ]:
# KNN
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder

data = df

# Chuyển đổi thuộc tính văn bản thành dạng số bằng LabelEncoder
label_encoder = LabelEncoder()
data['pc_brand'] = label_encoder.fit_transform(data['pc_brand'])
data['pc_model'] = label_encoder.fit_transform(data['pc_model'])
data['elt_condition'] = label_encoder.fit_transform(data['elt_condition'])
data['laptop_screen_size'] = label_encoder.fit_transform(data['laptop_screen_size'])
data['elt_warranty'] = label_encoder.fit_transform(data['elt_warranty'])
data['pc_cpu'] = label_encoder.fit_transform(data['pc_cpu'])
data['pc_vga'] = label_encoder.fit_transform(data['pc_vga'])

# Tách thành features (đặc trưng) và target (giá)
data = data.drop(['elt_origin'], axis=1)
X = data.drop('price', axis=1)
y = data['price']

# Tách dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Huấn luyện mô hình K-Nearest Neighbors
model = KNeighborsRegressor()
model.fit(X_train, y_train)

# Dự đoán giá trên tập kiểm tra
y_pred = model.predict(X_test)

# Đánh giá hiệu suất mô hình
r2 = r2_score(y_test, y_pred)

print(f"Coefficient of Determination (R^2): {r2}")

Coefficient of Determination (R^2): 0.523054928993286
